##ChatFeature

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import LLMChain


groq_api_key="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"

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0.5,
    api_key=groq_api_key
)

memory = ConversationBufferWindowMemory(
    window_size=10
)

system_prompts="""
Role & Personality:
You are a highly intelligent, resourceful, and conversational AI assistant. Your goal is to provide accurate, engaging, and helpful responses across various domains. You adapt to the user’s tone, preferences, and needs while maintaining clarity and professionalism.

Capabilities & Scope:
You can assist with:
 -General Knowledge – Provide well-researched, fact-based answers.
 -Creative Writing – Help with storytelling, scriptwriting, poetry, and content generation.
 -Problem-Solving – Offer logical reasoning, strategies, and step-by-step explanations.
 -Coding & Tech Support – Assist in programming, debugging, and optimization across multiple languages.
 -Productivity & Organization – Help plan schedules, manage tasks, and suggest tools for efficiency.
 -Personal Development – Provide insights on learning, motivation, habits, and self-improvement.
 -Health & Wellness – Share general fitness, mental health, and wellness tips (without medical diagnosis).
 -Business & Finance – Offer strategies, market insights, and entrepreneurship guidance.
 -Entertainment & Pop Culture – Engage in discussions on movies, books, music, and trends.

Interaction Style:

Be engaging and context-aware.
Adjust complexity based on user expertise (beginner-friendly or advanced).
When needed, break down responses into easy-to-follow steps.
Ask clarifying questions if the user’s request is vague.
Offer multiple perspectives when applicable.
Constraints & Ethics:

Avoid misinformation—always provide reliable, fact-checked responses.
Do not engage in harmful, unethical, or illegal topics.
Prioritize user privacy and confidentiality.
Special Instructions:

If a user asks for creative content, tailor the style based on their preferences.
For coding, provide optimized, well-commented, and structured code snippets.
If discussing technical topics, explain concepts in an easy-to-understand manner.
When offering advice, use "consider this approach" instead of being overly directive.
you are given {question} and you are asked to provide a response
"""

prompt = PromptTemplate(
    template=system_prompts,
    input_variables=["question"]
)

chain = LLMChain(
    prompt=prompt,
    memory=memory,
    llm=llm
)

def handle_qna(user_input):
    """
    Handles user queries for Q&A functionality.
    """
    try:
        response = chain.run(user_input)
        return response
    except Exception as e:
        return f"An error occurred while processing your question: {e}"



In [19]:
handle_qna("can you get when Champios tropy starts in cricket 2025")

"I'm glad you're asking about the Champions Trophy in cricket for 2025! However, as of the time I was last trained in 2021, the specific dates and details for the 2025 Champions Trophy have not been officially announced. I always strive to provide accurate and up-to-date information, but in this case, I don't have the exact details you're looking for. I recommend checking the official International Cricket Council (ICC) website or other reputable sports news sources closer to the event for the most accurate information."

##Managing Email

In [110]:
import smtplib
from email.message import EmailMessage
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import PromptTemplate
import re

def extract_email(input):
    # Regular expression pattern to match email addresses
    email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
    
    # Search for email in the text
    match = re.search(email_pattern, input)
    
    # Return the email if found, else None
    return match.group(0) if match else None



def compose_email(input):

    prompt = f"""

    Input: {input}
    Output:
        -Do not write Subject on above of email here
        -my name is Pankaj Kumar,my email is pankajkumarjnv76653@gmailcom contact 0019902
        -generate  an  clear consise and crisp email to given output
        -Just give email no extra things
        
    """

    messages = [
        SystemMessage(content="You are an email writing assistant"),
        HumanMessage(content=prompt)
    ]

    reponse=llm.invoke(messages)
    return reponse.content

def get_subject_of_email(input_text):
    prompt = PromptTemplate.from_template("Give me a good Subject for email about {input_text}. Just in  maximum 5 words")

    formatted_prompt = prompt.format(input_text=input_text)
    
    response = llm.invoke(formatted_prompt)
    
    return response.content




In [ ]:
import smtplib
from email.message import EmailMessage

# Step 1: Define Email Credentials
EMAIL_ADDRESS = "pk7372069@gmail.com"
EMAIL_PASSWORD = "sliet@37"

def send_email(to_email, subject, body):
    try:
        msg = EmailMessage()
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = to_email
        msg['Subject'] = subject
        msg.set_content(body)

        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            smtp.send_message(msg)

        return "Email sent successfully!"
    
    except Exception as e:
        return f"Error sending email: {e}"
    


In [2]:
send_email(
    to_email="pankajkumarjnv76653@gmail.com",
    subject="Regarding the upcoming meeting",
    body="Hello, I would like to discuss the agenda for our upcoming meeting. Please let me know your availability. Thank you."
)

"Error sending email: (535, b'5.7.8 Username and Password not accepted. For more information, go to\\n5.7.8  https://support.google.com/mail/?p=BadCredentials d9443c01a7336-220d5591e97sm117812645ad.244 - gsmtp')"

jondoe23@gmail.com
